In [2]:
# Library Imports
import pickle
import streamlit as st # Version 1.25.0
import pandas as pd # Version 2.0.1
import numpy as np # Version 1.3.0
import seaborn as sns # Version 0.12.2
import matplotlib.pyplot as plt # version 3.6.3
from sklearn.ensemble import GradientBoostingRegressor # Version 1.3.0

In [23]:
data1 = pd.read_csv("../Data/app_df.csv.gz", compression="gzip")

data1

,name,description,neighborhood_overview,host_listings_count,latitude,longitude,property_type,room_type,bathrooms,bedrooms,...,zipcode,median_income_dollars,property_type_binary,private,text,text_trunc,word_counts,sentiment,listing_url,review_score_weighted
0,Huge Oceanview decks+RooftopDeck☀sleeps 10☀Gar...,10 steps from the boardwalk! Beautiful beach h...,NaN,52.0,32.78568,-117.25271,Entire home,Entire home/apt,2.0,2.0,...,92109,95170.0,house,1,Huge Oceanview decks+RooftopDeck☀sleeps 10☀Gar...,Huge Oceanview decks+RooftopDeck☀sleeps 10☀Gar...,171,0.900577,https://www.airbnb.com/rooms/52582829,150.04
1,Quiet Bedroom in the heart of Pacific Beach,"Enjoy staying in the heart of Pacific Beach, S...",NaN,0.0,32.79801,-117.25443,Private room in condo,Private room,1.0,1.0,...,92109,95170.0,room,1,Quiet Bedroom in the heart of Pacific BeachEnj...,Quiet Bedroom in the heart of Pacific BeachEnj...,68,0.997769,https://www.airbnb.com/rooms/53085803,5.00
2,"Modern and Spacious home, 1 block from the Bay","Modern, Open, and Clean. Designed for flow, fu...",Just a few blocks away - Rocky's Pub ( Best Bu...,4.0,32.79184,-117.23468,Entire home,Entire home/apt,3.0,4.0,...,92109,95170.0,house,1,"Modern and Spacious home, 1 block from the Bay...","Modern and Spacious home, 1 block from the Bay...",179,0.999661,https://www.airbnb.com/rooms/53082401,45.00
3,"Entire 3B Beach House w/ Laundry, Parking and A/C",Welcome to steps from it all. Pacific Beach ho...,NaN,0.0,32.78846,-117.23962,Entire home,Entire home/apt,1.0,3.0,...,92109,95170.0,house,1,"Entire 3B Beach House w/ Laundry, Parking and ...","Entire 3B Beach House w/ Laundry, Parking and ...",55,0.999559,https://www.airbnb.com/rooms/53010441,NaN
4,Mission Beach Surf Van w/ Bed and Outlets,This is a van with a bed in it and a solar pow...,NaN,4.0,32.77687,-117.25122,Camper/RV,Entire home/apt,1.0,NaN,...,92109,95170.0,house,0,Mission Beach Surf Van w/ Bed and OutletsThis ...,Mission Beach Surf Van w/ Bed and OutletsThis ...,130,0.062897,https://www.airbnb.com/rooms/53011657,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18622,"Fletcher Hills - modern 1 BR.Easy, no step acc...",** 48 hours between check-ins. Thorough clean...,Residential neighborhood with nearby family an...,1.0,32.80724,-116.98927,Entire rental unit,Entire home/apt,1.0,1.0,...,92020,64691.0,house,1,"Fletcher Hills - modern 1 BR.Easy, no step acc...","Fletcher Hills - modern 1 BR.Easy, no step acc...",186,0.321106,https://www.airbnb.com/rooms/36210645,748.50
18623,"Amazing views, Spacious House, Large Enclosed ...",Enjoy the peaceful views from all rooms in thi...,NaN,4.0,32.79728,-116.98843,Entire home,Entire home/apt,2.0,3.0,...,92020,64691.0,house,1,"Amazing views, Spacious House, Large Enclosed ...","Amazing views, Spacious House, Large Enclosed ...",165,0.844013,https://www.airbnb.com/rooms/36210735,5.00
18624,San Diego East County Retreat,Enjoy a relaxing bedroom suite (2 rooms adjoin...,The home is located in a quiet neighborhood ca...,1.0,32.81523,-116.98643,Private room in home,Private room,1.0,1.0,...,92020,64691.0,room,1,San Diego East County RetreatEnjoy a relaxing ...,San Diego East County RetreatEnjoy a relaxing ...,190,0.999578,https://www.airbnb.com/rooms/35768403,156.16
18625,"2B Guest house, Private Door, parking. NO SMOKING",Welcome to our place in the peaceful Fletcher ...,"Quite, on top of a hill, beautiful view of a c...",1.0,32.79554,-116.98614,Entire guesthouse,Entire home/apt,1.0,2.0,...,92020,64691.0,house,1,"2B Guest house, Private Door, parking. NO SMOK...","2B Guest house, Private Door, parking. NO SMOK...",159,0.057779,https://www.airbnb.com/rooms/36209188,50.00


In [14]:
import re
url_match = re.compile(r"[\w\W]*\?")

url = re.match(r"[\w\W]*\?", "https://www.airbnb.com/rooms/50228360?adults=1&category_tag=Tag%3A7769&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=flex_destinations_search&check_in=2023-09-07&check_out=2023-09-14&source_impression_id=p3_1690775026_EM%2FlsHtKrNWan0kZ&previous_page_section_name=1000&federated_search_id=9deded69-3a20-4f10-b22a-474c3f4ecc1b").group(0).strip()[:-1]

In [86]:
y = data1.loc[data1["listing_url"] == url]
y = y[[x for x in y.columns.tolist() if x in data.columns.tolist()]]
y["property_type_binary"].values[0] == "room"
#y.shape

True

In [89]:
round(room_model.predict(room_pipeline.transform(y))[0],2)

c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


87.48

In [4]:
data.columns

Index(['host_listings_count', 'property_type', 'room_type', 'bathrooms',
       'bedrooms', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_maximum_nights', 'has_availability',
       'availability_30', 'availability_365', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
       'zipcode', 'median_income_dollars', 'property_type_binary', 'private',
       'sentiment', 'review_score_weighted'],
      dtype='object')

In [69]:
pd.DataFrame(room_pipeline.transform(pd.DataFrame(data.iloc[0]).T))

c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
0,-0.153462,1.11041,1.75293,-0.193201,-0.841116,-0.256698,-0.044842,1.012914,-0.660787,1.378007,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


In [3]:
# Read in Data
data = pd.read_csv("../Data/model_ready.csv.gz", compression = "gzip")

# Load models
house_model = pickle.load(open("../Code Library/Models/gb_house_tuned.sav", "rb"))
room_model = pickle.load(open("../Code Library/Models/gb_room_tuned.sav", "rb"))

# Load fitted pipelines
house_pipeline = pickle.load(open("../Deployment_Files/house_pipeline.sav", "rb"))
room_pipeline = pickle.load(open("../Deployment_Files/room_pipeline.sav", "rb"))

In [5]:
data

,host_listings_count,property_type,room_type,bathrooms,bedrooms,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_maximum_nights,...,calculated_host_listings_count,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,zipcode,median_income_dollars,property_type_binary,private,sentiment,review_score_weighted
0,52.0,Entire home,Entire home/apt,2.0,2.0,728.0,2,28,1.0,1125.0,...,30,0,0,3.77,92109,95170.0,house,1,0.900577,150.04
1,0.0,Private room in condo,Private room,1.0,1.0,70.0,7,30,7.0,30.0,...,1,1,0,0.15,92109,95170.0,room,1,0.997769,5.00
2,4.0,Entire home,Entire home/apt,3.0,4.0,595.0,3,365,3.0,365.0,...,3,0,0,1.53,92109,95170.0,house,1,0.999661,45.00
3,0.0,Entire home,Entire home/apt,1.0,3.0,926.0,2,14,2.0,1125.0,...,2,0,0,0.00,92109,95170.0,house,1,0.999559,0.00
4,4.0,Camper/RV,Entire home/apt,1.0,1.0,62.0,1,1125,1.0,1125.0,...,3,1,0,0.12,92109,95170.0,house,0,0.062897,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18489,1.0,Entire rental unit,Entire home/apt,1.0,1.0,96.0,4,1125,4.0,1125.0,...,1,0,0,2.30,92020,64691.0,house,1,0.321106,748.50
18490,4.0,Entire home,Entire home/apt,2.0,3.0,262.0,1,365,1.0,365.0,...,3,0,0,1.00,92020,64691.0,house,1,0.844013,5.00
18491,1.0,Private room in home,Private room,1.0,1.0,50.0,2,30,2.0,30.0,...,1,1,0,0.53,92020,64691.0,room,1,0.999578,156.16
18492,1.0,Entire guesthouse,Entire home/apt,1.0,2.0,175.0,1,365,1.0,365.0,...,1,0,0,4.23,92020,64691.0,house,1,0.057779,50.00


In [72]:
data[["host_listings_count", "calculated_host_listings_count"]].head(50)

,host_listings_count,calculated_host_listings_count
0,52.0,30
1,0.0,1
2,4.0,3
3,0.0,2
4,4.0,3
5,3.0,2
6,5.0,5
7,0.0,2
8,3.0,2
9,8.0,8


In [40]:
test_dict = data.set_index("zipcode").to_dict()["median_income_dollars"]
test_dict[92127]

156464.0

In [70]:
data.loc[(data["zipcode"] == int(92127)) 
        & (data["property_type_binary"] == "house")]["instant_bookable"].mode()

0    f
Name: instant_bookable, dtype: object

In [18]:
type = "Entire home/apt"
property_type_binary_feat21 = "room" if "room" in type.lower() else "house"
property_type_binary_feat21

'house'